In [1]:
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms

import pandas as pd

import time
import os
import PIL.Image as Image
from IPython.display import display

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


## Load the data and transform

First, lets create some transforms for our data and load the train/test data+labels from the folders.

Here we are using 300x300 images with random horizontal flip, random rotation and normalization

In [2]:
dataset_dir = "../input/car_data/car_data/"

train_transform = transforms.Compose([transforms.Resize((300, 300)),
                                 transforms.RandomRotation(15),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
test_transform = transforms.Compose([transforms.Resize((300, 300)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

dataset = torchvision.datasets.ImageFolder(root=dataset_dir+"train", transform = train_transform)
trainloader = torch.utils.data.DataLoader(dataset, batch_size = 32, shuffle=True, num_workers = 2)

dataset2 = torchvision.datasets.ImageFolder(root=dataset_dir+"test", transform = test_transform)
testloader = torch.utils.data.DataLoader(dataset2, batch_size = 32, shuffle=False, num_workers = 2)


## Model training function

Here we train our model, after each epoch, we test the model on the test data to see how it's going

In [3]:
def train_model(model, criterion, optimizer, scheduler, n_epochs = 5):
    
    losses = []
    accuracies = []
    
    model.train()
    for epoch in range(n_epochs):
        since = time.time()
        running_loss = 0.0
        running_correct = 0.0
        counter = 0
        for i, data in enumerate(trainloader, 0):
            
            inputs, labels = data
            
            # Assign data to CUDA if available
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            counter += len(inputs)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels) # We need to give the original propabilities instead of the final labels.
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            running_correct += (labels==predicted).sum().item()

        epoch_duration = time.time()-since
        epoch_loss = running_loss / counter
        print(running_correct, counter)
        epoch_acc = 100 * (running_correct / counter)
        print("Epoch %s, duration: %d s, loss: %.4f, acc: %.4f" % (epoch+1, epoch_duration, epoch_loss, epoch_acc))
        
        losses.append(epoch_loss)
        accuracies.append(epoch_acc)
        
        scheduler.step(epoch_acc)
        since = time.time()
    print('Finished Training')
    
    return model, losses, accuracies

### Evaluate on testing data
This function is called out after training on the training data. We then measure the accuracy of the model.

In [4]:
def eval_model(model):
    correct = 0.0
    total = 0.0
    with torch.no_grad():
        for i, data in enumerate(testloader, 0):
            images, labels = data
            
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model_ft(images)
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_acc = 100.0 * (correct / total)
    print('Accuracy of the network on the test images: %d %%' % (
        test_acc))
    return test_acc


def eval_model_for_plot(model):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    num_samples = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            num_samples += inputs.size(0)

            all_preds.append(preds.cpu().numpy().tolist())
            all_labels.append(labels.data.cpu().numpy())

        test_loss = running_loss / num_samples
        test_accuracy = running_corrects.double() / num_samples

    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)
    accuracy = (all_preds == all_labels).astype(np.float32).mean()

    return test_loss, test_accuracy, accuracy, all_preds, all_labels



In [5]:
model_ft = models.densenet161(pretrained=True)

Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.torch/models/densenet161-8d451a50.pth
115730790it [00:01, 82133705.17it/s]


In [6]:

num_ftrs = model_ft.classifier.in_features

# replace the last fc layer with an untrained one (requires grad by default)
model_ft.classifier = nn.Linear(num_ftrs, 196)
model_ft = model_ft.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_ft.parameters(), lr=0.01, momentum=0.9)

lrscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, threshold = 0.9)

In [ ]:
model_ft, training_losses, training_accs, test_accs = train_model(model_ft, criterion, optimizer, lrscheduler, n_epochs=10)

In [ ]:
torch.save(model_ft.state_dict(), './AlexNetFineTurning')

### As we can see the model reached 99% training accuracy by epoch 10.


In [ ]:
# Plot the training accuracy
plt.plot(training_accs)
plt.title('Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot the training losses
plt.plot(training_losses, label='Training loss')
plt.title('Training Losses')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
f, axarr = plt.subplots(2,2, figsize = (12, 8))
axarr[0, 0].plot(training_losses)
axarr[0, 0].set_title("Training loss")
axarr[0, 1].plot(training_accs)
axarr[0, 1].set_title("Training acc")
axarr[1, 0].plot(test_accs)
axarr[1, 0].set_title("Test acc")

In [ ]:
model_ft.eval()

In [ ]:
eval_model(model_ft)

In [ ]:
test_loss, test_acc, accuracy, all_preds, all_labels = eval_model_for_plot(model_ft)

In [ ]:
test_loss

In [ ]:
test_acc

In [ ]:
# Create scatter plot
plt.scatter(np.arange(len(all_preds)), all_preds == all_labels, s=5)
plt.title('Accuracy of each test image')
plt.xlabel('Index of test image')
plt.ylabel('Accuracy')
plt.show()

### View the missclassified Images

In [ ]:
import pandas as pd

names = list(pd.read_csv("../input/names.csv", names = ['names'])['names'])
names.sort()

In [ ]:
from matplotlib.pyplot import figure

def getMisclassifiedImages(model):
    count = 0
    with torch.no_grad():
        fig = figure(figsize=(10, 10))
        for i, data in enumerate(testloader, 0):
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model_ft(images)
            _, predicted = torch.max(outputs.data, 1)
            
            for i in range(0, len(predicted)):
                if (predicted[i] != labels[i]):
                    img = images[i].cpu().numpy().transpose((1, 2, 0))
                    mean = np.array([0.485, 0.456, 0.406])
                    std = np.array([0.229, 0.224, 0.225])
                    img = std * img + mean
                    plt.subplot(3, 2 , count + 1)
                    plt.tight_layout()
                    plt.title("Predicted : " + str(names[predicted[i]]) + "\n" + "Actual : " + str(names[labels[i]]) + "\n")
                    plt.imshow((img * 255).astype(np.uint8))
                    plt.axis("off")
                    count += 1
                    if (count == 6):
                        break
            if (count == 6):
                break

In [ ]:
getMisclassifiedImages(model_ft)

In [ ]:
nb_classes = 196

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(testloader):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

In [ ]:
import pandas as pd
import seaborn as sns
nb_classes = 196
confusion_matrix = np.zeros((nb_classes, nb_classes))
with torch.no_grad():
    for i, (inputs, classes) in enumerate(testloader):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

plt.figure(figsize=(20,20))


df_cm = pd.DataFrame(confusion_matrix)
heatmap = sns.heatmap(df_cm, annot=True,fmt="d")

heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right',fontsize=15)
heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right',fontsize=15)

plt.ylabel('True label', fontsize=15)
plt.xlabel('Predicted label', fontsize=15)
plt.show()